## Label and Add Feature Set to DataFrame of Opinions

In [2]:
import io
import pandas as pd
import pickle

In [5]:
# Open the dataframe
infile = open('df_clean.data', 'rb')
df = pickle.load(infile)
infile.close()

In [6]:
df.describe()

,Opinion
count,3922
unique,3922
top,p an unpublished opinion of the north carolina...
freq,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3922 entries, 0 to 3949
Data columns (total 1 columns):
Opinion    3922 non-null object
dtypes: object(1)
memory usage: 61.3+ KB


In [21]:
df.Opinion[3500]

'an unpublished opinion of the north carolina court of appeals does not constitute controlling legal authority. citation is disfavored, but may be permitted in accordance with the provisions of rule 30(e)(3) of the north carolina rules of appellate procedure. no. coa02-1669 north carolina court of appeals filed: 4 november 2003 douglas james swinton, ancillary administrator of the estate of ryan a. swinton, douglas james swinton, individually, and carol ann swinton, individually, plaintiffs, v. la fogata corporation, defendant. dare county no. 01-cvs-452 appeal by plaintiffs from order entered 30 september 2002 by judge william c. griffin, jr., in dare county superior court. heard in the court of appeals 15 september 2003. aycock & butler, pllc, by betsy butler; and aldridge, seawell, spence & felthousen, by w. mark spence, for plaintiff appellants. stephenson & stephenson, llp, by james b. stephenson ii, and dena white waters, for defendant appellee. mccullough, judge. plaintiffs appe

# create tagging and feature set 
# create models 
# create pipeline
# create dashboard


There are many great paid proxy providers like Smartproxy, Netnut, Highproxies, etc. that can be used when scraping and at least you will know that you will get high-quality services that are safe to use with lots of features and big pools of IPs that you can use.

In [ ]:
# Open the dataframe
(fix the name)
# infile = open('pdf_addresses.data','rb')
# pdf_addresses = pickle.load(infile)
# infile.close()